In [58]:
import json
import requests
import pandas as pd
import os

In [59]:
OLLAMA_NODE = "arctrdagn032" # TODO: Change this variable to the node where Ollama is running
BASE_URL = f"http://{OLLAMA_NODE}:11434/api/chat"
model = "gemma3-optimized:27b" # TODO: Change this variable to the model you want to use

In [60]:
id_transcript = "/data/users4/xli/ai-psychiatrist/datasets/daic_woz_dataset/358_P/358_TRANSCRIPT.csv"

transcript_data = pd.read_csv(id_transcript, sep='\t')
dialogue_txt = transcript_data.dropna(subset =['speaker', 'value'])
dialogue_txt['dialogue'] = dialogue_txt['speaker'] +  " : " + dialogue_txt['value']

full_dialogue = "\n".join(dialogue_txt['dialogue'])

print(full_dialogue[:1000])

Ellie : hi i'm ellie thanks for coming in today
Ellie : i was created to talk to people in a safe and secure environment 
Ellie : i'm not a therapist but i'm here to learn about people and would love to learn about you i'll ask a few questions to get us started
Ellie : and please feel free to tell me anything your answers are totally confidential
Ellie : are you okay with this
Participant : yes
Ellie : so how are you doing today  
Participant : fine
Ellie : where are you from originally 
Participant : tampa florida 
Ellie : why'd you move to l_a
Participant : uh at that point i wanted to get into the entertainment industry 
Ellie : when'd you move to l_a
Participant : um nineteen ninety 
Ellie : how easy was it for you to get used to living in l_a
Participant : it took a long time to get used to it it wasn't easy 
Ellie : what are some things you really like about l_a
Participant : well the weather and that if you have a hobby you could always find a large group of people that are inte

In [ ]:
qual_assessment = f"""
The following is an interview transcript from a psychiatric assessment of a subject who is being screened for depression. Please note that 'Ellie' is the interviewer, and 'Participant' is the subject being assessed.
{full_dialogue}

Please:
1. Provide an overall qualitative assessment of the subject’s mental health. 
2. Summarize social aspects that may influence the subject’s mental health.
3. biological aspects that may influence the subject’s mental health.
4. Identify potential risk factors the subject may be experiencing.

Output should be formatted as bullet points with headings for each section using stars. Example: **Tiredness** <explanation of tiredness>. Do not include any additional text outside the bullet points
Please answer in this XML format with each tag on a new line, properly indented. Use straight quotes instead of curly quotes, and do not include any additional text outside the XML tags:

<assessment>
<!-- Summary of participant’s overall mental health -->
</assessment>


<social_factors>
<!-- Summary of social influences on patient's health -->
</social_factors>


<biological_factors>
<!-- Summary of biological influences on patient's health -->
</biological_factors>


<risk_factors>
<!-- Summary of potential risk factors -->
</risk_factors>
"""

response = requests.post(
  BASE_URL,
  json = {
    "model": model,
    "messages": [{"role": "user", "content": qual_assessment}],
    "stream": False
  }
)

content = response.json()['message']['content']
print(content)

```xml
<assessment>
The participant presents with a complex mental health history, marked by a prior diagnosis of dysthymia (a chronic, mild form of depression) and current symptoms suggestive of ongoing mood disturbance. They report feeling irritable, moody, and tired, along with difficulty sleeping and experiencing periods where they feel “airheaded” and slow. While they claim to be “good” at controlling their temper and state they don't argue much, their responses regarding frustration and negative people are concerning. Their recall of recent happy experiences is poor, and they struggle to pinpoint when they last felt genuinely happy.  The participant displays some awareness of their emotional state and a desire for self-improvement (advice to younger self). However, they also exhibit a tendency to ruminate and a history of being affected by negative interactions with others. The participant's communication style is somewhat rambling and tangential at times, which could indicate di

In [62]:
quant_assessment = f"""
The following is an interview transcript from a psychiatric assessment of a subject who is being screened for depression. Please note that 'Ellie' is the interviewer, and 'Participant' is the subject being assessed.
{full_dialogue}

Please:
1. Predict each score based on the patient's responses in the interview transcript.Provide a score for each of the following scale. Score from 0-3 (0- not at all 1- several days 2- more than half the days 3- nearly every day):

    a) little interest or pleasure in doing things 
    b) Feeling down, depressed, or hopeless
    c) Trouble falling or staying asleep, or sleeping too much
    d) Feeling tired or having little energy
    e) Poor appetite or overeating
    f) Feeling bad about yourself - or that you are a failure or have let yourself or your family down
    g) Trouble concentrating on things, such as reading the newspaper or watching television
    h) Moving or speaking so slowly that other people could have noticed. Or the opposite - being so fidgety or restless that you have been moving around a lot more than usual

2. Provide a brief explanation for each score.
3. Provide a summary of the participant's mental health based on the scores.
4. Provide a total score based on the sum of the individual scores, along with the category the total score falls into:
    0 - 4: no significant depressive symptoms
    5 - 9: mild depressive symptoms
    10 - 14: moderate depressive symptoms
    15 - 19: moderately severe depressive symptoms
    20 - 24: severe depressive symptoms
    

Please answer in this XML format with each tag on a new line, properly indented. Use straight quotes instead of curly quotes, and do not include any additional text outside the XML tags.:

<assessment>
    <a>
        <!--little interest or pleasure in doing things-->
        <value></value>
        <explanation></explanation>
    </a>
    <b>
        <!--Feeling down, depressed, or hopeless-->
        <value></value>
        <explanation></explanation>
    </b>
    <c>
        <!--Trouble falling or staying asleep, or sleeping too much-->
        <value></value>
        <explanation></explanation>
    </c>
    <d>
        <!--Feeling tired or having little energy-->
        <value></value>
        <explanation></explanation>
    </d>
    <e>
        <!--Poor appetite or overeating-->
        <value></value>
        <explanation></explanation>
    </e>
    <f>
        <!--Feeling bad about yourself - or that you are a failure or have let yourself or your family down-->
        <value></value>
        <explanation></explanation>
    </f>
    <g>
        <!--Trouble concentrating on things, such as reading the newspaper or watching television-->
        <value></value>
        <explanation></explanation>
    </g>
    <h>
        <!--Moving or speaking so slowly that other people could have noticed. Or the opposite - being so fidgety or restless that you have been moving around a lot more than usual-->
        <value></value>
        <explanation></explanation>
    </h>
    
    <!-- Total score -->
    <total_score></total_score>
    <category></category>
    
    <!-- Summary of participant's mental health based on scores --> 
</assessment>
"""

response = requests.post(
  BASE_URL,
  json = {
    "model": model,
    "messages": [{"role": "user", "content": quant_assessment}],
    "stream": False
  }
)

content = response.json()['message']['content']
print(content)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
overall_eval = f"""
You are an AI psychiatrist. Based on the qualitative and quantitative assessments below, provide an overall evaluation of the patient's mental health.

Qualitative Assessment:
{qual_assessment}

Quantitative Assessment:
{quant_assessment}


Please:

1. Provide a summary of the patient's mental health.
2. Provide a brief explanation of the patient's mental health status based on the qualitative and quantitative assessments.
3. Provide the total score from the PHQ-8 assessment, the sum of individual scores
4. Provide the category as one of: None, Mild, Moderate, Moderately Severe, Severe

Please provide your evaluation in the following XML format, with each tag on a new line, properly indented. Use straight quotes instead of curly quotes, and do not include any additional text outside the XML tags:

<evaluation>
    <summary>
        <!-- Summary of the patient's mental health -->
    </summary>
    <explanation>
        <!-- Brief explanation of the patient's mental health status -->
    </explanation>
    <total_score>
        <!-- Total score from the PHQ-8 assessment -->
    </total_score>
    <category>
        <!-- Category of the total score -->
    </category>
</evaluation>
"""

response = requests.post(
  BASE_URL,
  json = {
    "model": model,
    "messages": [{"role": "user", "content": overall_eval}],
    "stream": False
  }
)

content = response.json()['message']['content']
print(content)

SyntaxError: incomplete input (1631609681.py, line 1)